In [ ]:
#Goal: Make the script for MMPBSA2LIE for nos and other stuff 
#this is adapted/copied from 210907MMPBSA2lie to accomodate for the sanders version of MMPBSA 

Basic Information: 
[ Zhou Et Al 1994 ] 
- DGcalc = Beta(<Uele>lig-prot - <Uele>lig-wat) + Alpha ( <Uvdw>lig-prot - <Uvdw>lig-wat) + gamma 

In [ ]:
#paperwork/documenting 

#!/usr/bin/env python
"""
Purpose: Convert MMPBSA Free energy data results to get LIE for NOS proteins 

Usage: python mmpbsa2lie.py - ib [boundfile.dat] -il [ligandfreefile]  -o [file.out]
[optional] -input [inputtype] -a [alpha value] -b [beta value] -g [gamma value]
 
By: Oanh TN Tran 
Version: 0.01, started: Sept 10, 2021 

"""

##[description]
"""
    Get energy from input file.
    Parameters
    ----------
    inputfile : str
        Name of input file. Files include: .dat , .____ , .____ , 
    outputfile: str
        The name of output file. Extension include: .lie, .txt, .out 
    a : float 
        Alpha values, Alpha = 0.18 unless stated otherwise 
    b : float 
        Beta values, Beta =0.5 unless stated otherwise         
    g : float 
        Gamma values, gamma = 0 unless stated otherwise 
    Returns
    -------
    output file:  
        _____. [put extensions here]
    """

In [ ]:
#import specific modules:
import matplotlib 
import csv
import os 
import numpy 

 ----------
## Version 3 - Adapted, mmpbsa sanders 

---------- 

In [9]:
#Version 3 #lie- linear interactive energy 
#divide section into chunks and create dictionary of dictionary  

def main(infile, outfile,  a= 0.18, b= 0.5, g= 0): #default unless stated otherwise 
        
    #reading input file, identifying type of data (GB or PB), and get specific value  
    with open (infile, 'r') as mmpbsafile:
        mmpbsaread = mmpbsafile.readlines() 

    componentname = ["COMPLEX", "RECEPTOR", "LIGAND", "DELTA"]
    MMdict = {} 

    ##checking if the file has listed qualities: 
    for i in range(len(mmpbsaread)):
        if componentname[0] in mmpbsaread[i]:
            titleline=i+4
        elif componentname[3] in mmpbsaread[i]:
            deltaline=i+4
            break 
    #make empty dictionary 
    for i in range(len(componentname)):
        MMdict[componentname[i]] = {} 
    
    #make dictionary of the complex, receptor, ligand   
    for i in range(titleline, titleline+8):
        splitrow = mmpbsaread[i].split()
        MMdict[componentname[0]][splitrow[0]] = float(splitrow[1])
        MMdict[componentname[1]][splitrow[0]] = float(splitrow[3])
        MMdict[componentname[2]][splitrow[0]] = float(splitrow[5])
    
    #make dictionary of the delta 
    for i in range(deltaline,deltaline+8): #till the end 
        splitrow = mmpbsaread[i].split()
        MMdict[componentname[3]][splitrow[0]] = splitrow[1]

    #print ("Done making MMdict") #check point 
    #Identify important object: 
    Ulig_protBound = MMdict['COMPLEX']['ELE'] - MMdict["RECEPTOR"]["ELE"] 
    Ulig_watBound = MMdict["LIGAND"]["GBSOL"]  
    Ulig_watFree = MMdict["LIGAND"]["GB"] + MMdict["LIGAND"]["GBSUR"]

    Dele = Ulig_protBound + 2*Ulig_watBound - 2*Ulig_watFree
    
    Dvdw = MMdict["COMPLEX"]["VDW"] - MMdict["RECEPTOR"]["VDW"] 
    
    DGlie = b*(Dele) + a*(Dvdw) + g 

    #setting up the output file, write the begining part of the mmgbsaread file up until GENERALIZED BORN
    with open (outfile, 'w') as outputfile: 
        LIEresult = """

Generated with mmpbsa2lie.py 
-------------------------------------------------------------------------------

The LIE value for %s is: 
Dele = %0.2f
Dvdw = %0.2f
DGLie = %0.2f 

-------------------------------------------------------------------------------
------------------------------------------------------------------------------- 
""" %(infile, Dele, Dvdw, DGlie) ## fix the % to assigment 
    
    outputfile.write(LIEresult)
    
    Print("Done with %s" %inputfile_dir)
    
    #closing read/write file 
    mmpbsafile.close()
    outputfile.close() 
        
    return DGlie  #What do i put in the output file? 


 -----
## Parsing
----- 

In [13]:
#parsin for help arguments
 

if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()

    parser.add_argument("-i", "--infile",
            help="Input MMPBSA file")
    parser.add_argument("-o", "--outfile",
            help="Output LIE file")
    parser.add_argument("-a", "--alpha value",
            help="input alpha value, default is 0.18")
    
    ### added this version in the script for default version 
    #parser.add_argument("-a", "--alpha value",  nargs='?', const=1, type=float, default=0.18,
    #        help="input alpha value, default is 0.18")
    
    parser.add_argument("-b", "--beta value",
            help="input beta value, default is 0.5")
    parser.add_argument("-g", "--gamma value",
            help="input gamma value, default is 0")

    args = parser.parse_args()
    opt = vars(args.infile)
    
   
    main(**opt) #works with def main(**kwargs)
 

usage: ipykernel_launcher.py [-h] [-i INFILE] [-o OUTFILE] [-a ALPHA VALUE]
                             [-b BETA VALUE] [-g GAMMA VALUE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/otran/Library/Jupyter/runtime/kernel-285c32de-2618-49e0-9b6f-55822026f908.json


SystemExit: 2

some example: I found 
**kwargs - > dictionary https://www.w3schools.com/python/gloss_python_function_arbitrary_keyword_arguments.asp 
def myfunc(a,b, *args, **kwargs):
        a = kwargs.get('a', None)
        b = kwargs.get('b', None)
        g = k
      #etc
    myfunc(a,b, c='nick', d='dog', ...) 
   

 --------------
## Example of what MMGBSA input file looks like 
------------- 

In [6]:
#seeing an example file: 
with open ('/Users/otran/Desktop/mmpbsa2lie/stats421_100frames', 'r') as resultfile:
    #result_rdlines = resultfile.readlines()
    #print (result_rdlines)
        
    result_rd = resultfile.read()
    print (result_rd)
    #a = result_rd.find("POISSON BOLTZMANN:")
    #print (a)
    
resultfile.close() 

#                  COMPLEX                RECEPTOR                  LIGAND        
#          ----------------------- ----------------------- -----------------------
#                  MEAN        STD         MEAN        STD         MEAN        STD
#          ======================= ======================= =======================
ELE            -6898.00      43.58     -6818.19      42.83         6.08       1.05
VDW            -4264.58      54.76     -4226.27      53.32         7.24       1.49
INT            19774.91      77.07     19710.61      76.17        64.29       5.95
GAS             8612.32      91.43      8666.15      90.72        77.61       5.75
GBSUR            167.72       3.81       168.64       3.70         2.83       0.04
GB             -2368.98      43.81     -2416.31      43.62       -39.27       0.59
GBSOL          -2201.26      41.21     -2247.67      41.10       -36.43       0.58
GBELE          -9266.98      12.56     -9234.50      12.52       -33.18       0.95
GBTO

In [7]:
with open ('/Users/otran/Desktop/mmpbsa2lie/stats421_protein0100frames', 'r') as protfile:
    #result_rdlines = resultfile.readlines()
    #print (result_rdlines)
        
    prot_rd = protfile.read()
    print (prot_rd)

#                  COMPLEX                RECEPTOR                  LIGAND        
#          ----------------------- ----------------------- -----------------------
#                  MEAN        STD         MEAN        STD         MEAN        STD
#          ======================= ======================= =======================
ELE            -6898.00      43.58         6.08       1.05         6.08       1.05
VDW            -4264.58      54.76     -4264.58      54.76         7.24       1.49
INT            19774.91      77.07     19774.91      77.07        64.29       5.95
GAS             8612.32      91.43     15516.41      80.22        77.61       5.75
GBSUR            167.72       3.81       167.72       3.81         2.83       0.04
GB             -2368.98      43.81       -33.54       1.00       -39.27       0.59
GBSOL          -2201.26      41.21       134.18       4.11       -36.43       0.58
GBELE          -9266.98      12.56       -27.45       1.47       -33.18       0.95
GBTO

 ------
## Final MMPBSA2LIE.py script
-------- 

def main(**kwargs):
    #infile, outfile,  a= 0.18, b= 0.5, g= 0 (5 value)  #default unless stated otherwise
    """
    Purpose: Convert MMPBSA Free energy data results to get LIE for NOS proteins

    Usage: python mmpbsa2lie.py - i [file.dat]  -o [file.out]
    [optional] -a [alpha value] -b [beta value] -g [gamma value]
     
    By: Oanh TN Tran
    Version: 1.0, started: Sept 10, 2021

------------------- DESCRIPTION --------------------------------------------------------------

    Get energy from input file.
    Parameters
    ----------
    inputfile : str
        Name of input file.
    outputfile: str
        The name of output file. Extension include: .lie, .txt, .out
    a : float
        Alpha values, Alpha = 0.18 unless stated otherwise
    b : float
        Beta values, Beta =0.5 unless stated otherwise
    g : float
        Gamma values, gamma = 0 unless stated otherwise
    Returns
    -------
    output file:
        outputfile: gives Dele, Dvdw, DGlie values in a doc
    """
        
    #reading input file, identifying type of data (GB or PB), and get specific value
    with open (args.infile, 'r') as mmpbsafile:
        mmpbsaread = mmpbsafile.readlines()

    componentname = ["COMPLEX", "RECEPTOR", "LIGAND", "DELTA"]
    MMdict = {}

    ##checking if the file has listed qualities:
    for i in range(len(mmpbsaread)):
        if componentname[0] in mmpbsaread[i]:
            titleline=i+4
        elif componentname[3] in mmpbsaread[i]:
            deltaline=i+4
            break
    #make empty dictionary
    for i in range(len(componentname)):
        MMdict[componentname[i]] = {}
    
    #make dictionary of the complex, receptor, ligand
    for i in range(titleline, titleline+8):
        splitrow = mmpbsaread[i].split()
        MMdict[componentname[0]][splitrow[0]] = float(splitrow[1])
        MMdict[componentname[1]][splitrow[0]] = float(splitrow[3])
        MMdict[componentname[2]][splitrow[0]] = float(splitrow[5])
    
    #make dictionary of the delta
    for i in range(deltaline,deltaline+8): #till the end
        splitrow = mmpbsaread[i].split()
        MMdict[componentname[3]][splitrow[0]] = splitrow[1]

    #print ("Done making MMdict") #check point
    #Identify important object:
    Ulig_protBound = MMdict['COMPLEX']['ELE'] - MMdict['RECEPTOR']['ELE']
    Ulig_watBound = MMdict['LIGAND']['GBSOL']
    Ulig_watFree = MMdict['LIGAND']['GB'] + MMdict['LIGAND']['GBSUR']

    Dele = Ulig_protBound + 2*Ulig_watBound - 2*Ulig_watFree
    
    Dvdw = MMdict['COMPLEX']['VDW'] - MMdict['RECEPTOR']['VDW']
    
    DGlie = args.beta*(Dele) + args.alpha*(Dvdw) + args.gamma

    #setting up the output file, write the begining part of the mmgbsaread file up until GENERALIZED BORN
    with open (args.outfile, 'w') as outputfile:
        LIEresult = """

Generated with mmpbsa2lie.py
-------------------------------------------------------------------------------

The LIE value for %s is:
Dele = %0.2f
Dvdw = %0.2f
DGLie = %0.2f

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
""" %(args.infile, Dele, Dvdw, DGlie) ## fix the % to assigment
    
        outputfile.write(LIEresult)
    
    print("Done with %s" %args.infile)
    
    #closing read/write file
    mmpbsafile.close()
    outputfile.close()
        
    return DGlie  #What do i put in the output file How do I just get return value

if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()

    parser.add_argument("-in", "--infile",
            help="Input MMPBSA file")
    parser.add_argument("-o", "--outfile",
            help="Output LIE file")
    parser.add_argument("-a", "--alpha",  nargs='?', const=1, default=0.18, type=float,
            help="input alpha value, default is 0.18")
    parser.add_argument("-b", "--beta",  nargs='?', const=1, default=0.5, type=float,
            help="input beta value, default is 0.5")
    parser.add_argument("-g", "--gamma",  nargs='?', const=1, default=0, type=float,
            help="input gamma value, default is 0")

    args = parser.parse_args()
    opt = vars(args)
    
    #1main(args.infile, args.outfile) # might need to add a, b, g value
    
    main(**opt)
 


 ---------
## Edited version 1.1 (9/28/2021)
------- 

In [ ]:
def main(**kwargs):
    #infile, outfile,  a= 0.18, b= 0.5, g= 0 (5 value)  #default unless stated otherwise
    """
    Purpose: Convert MMPBSA Free energy data results to get LIE for NOS proteins

    Usage: python mmpbsa2lie.py -i [infilefile] -ip [protein_0file] -o [file.out]
    [optional] -a [alpha value] -b [beta value] -g [gamma value]
     
    By: Oanh TN Tran
    Version: 1.1, started: Sept 28, 2021

------------------- DESCRIPTION --------------------------------------------------------------

    Get energy from input file.
    Parameters
    ----------
    inputfile : str
    
    
    outputfile: str
        The name of output file. Extension include: .lie, .txt, .out
    a : float
        Alpha values, Alpha = 0.18 unless stated otherwise
    b : float
        Beta values, Beta =0.5 unless stated otherwise
    g : float
        Gamma values, gamma = 0 unless stated otherwise
    Returns
    -------
    output file:
        outputfile: gives Dele, Dvdw, DGlie values in a doc
    """
    
    def makedict (infile): 
    
        #reading input file, identifying type of data (GB or PB), and get specific value
        with open (infile, 'r') as mmpbsafile:
            mmpbsaread = mmpbsafile.readlines()

        componentname = ["COMPLEX", "RECEPTOR", "LIGAND", "DELTA"]
        MMdict = {}

        ##checking if the file has listed qualities:
        for i in range(len(mmpbsaread)):
            if componentname[0] in mmpbsaread[i]:
                titleline=i+4
            elif componentname[3] in mmpbsaread[i]:
                deltaline=i+4
                break
        #make empty dictionary
        for i in range(len(componentname)):
            MMdict[componentname[i]] = {}

        #make dictionary of the complex, receptor, ligand
        for i in range(titleline, titleline+9):
            splitrow = mmpbsaread[i].split()
            MMdict[componentname[0]][splitrow[0]] = float(splitrow[1])
            MMdict[componentname[1]][splitrow[0]] = float(splitrow[3])
            MMdict[componentname[2]][splitrow[0]] = float(splitrow[5])

        #make dictionary of the delta
        for i in range(deltaline,deltaline+9): #till the end
            splitrow = mmpbsaread[i].split()
            MMdict[componentname[3]][splitrow[0]] = splitrow[1]

        #ensure closing the file 
        mmpbsafile.close()

        return MMdict
    
    #Make a dictionary of the two files 
    MMdict = makedict(args.ligfreefile)
    protein_0 = makedict(args.boundfile)
    
    
    ##change the rest## 
    #Identify important object:
    Ulig_protBound = MMdict['COMPLEX']['ELE'] - MMdict['RECEPTOR']['ELE']
    Ulig_watBound =  protein_0['LIGAND']['GBSOL']
    Ulig_watFree = MMdict['LIGAND']['GB'] + MMdict['LIGAND']['GBSUR']

    Dele = Ulig_protBound + 2*Ulig_watBound - 2*Ulig_watFree
    
    Dvdw = MMdict['COMPLEX']['VDW'] - MMdict['RECEPTOR']['VDW']
    
    DGlie = args.beta*(Dele) + args.alpha*(Dvdw) + args.gamma

    #setting up the output file, write the begining part of the mmgbsaread file up until GENERALIZED BORN
    with open (args.outfile, 'w') as outputfile:
        LIEresult = """

Generated with mmpbsa2lie.py
-------------------------------------------------------------------------------

The LIE value for %s is:
Dele = %0.2f
Dvdw = %0.2f
DGLie = %0.2f

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
""" %(args.boundfile, Dele, Dvdw, DGlie) ## fix the % to assigment
    
        outputfile.write(LIEresult)
    
    print("Done with %s" %args.boundfile)
    
    #closing read/write file
    outputfile.close()
        
    return DGlie  #What do i put in the output file How do I just get return value

if __name__ == "__main__":
    
    import argparse
    parser = argparse.ArgumentParser()

    parser.add_argument("-i", "--infile",
            help="Input bound MMPBSA file")
    parser.add_argument("-ip", "--proteinfile",
            help="Input ligand-free MMPBSA file")
    parser.add_argument("-o", "--outfile",
            help="Output LIE file")
    parser.add_argument("-a", "--alpha",  nargs='?', const=1, default=0.18, type=float,
            help="input alpha value, default is 0.18")
    parser.add_argument("-b", "--beta",  nargs='?', const=1, default=0.5, type=float,
            help="input beta value, default is 0.5")
    parser.add_argument("-g", "--gamma",  nargs='?', const=1, default=0, type=float,
            help="input gamma value, default is 0")

    args = parser.parse_args()
    opt = vars(args)
    
    #1main(args.infile, args.outfile) # might need to add a, b, g value
    
    main(**opt)

In [22]:
def makedict (infile): 
    
    #reading input file, identifying type of data (GB or PB), and get specific value
    with open (infile, 'r') as mmpbsafile:
        mmpbsaread = mmpbsafile.readlines()

    componentname = ["COMPLEX", "RECEPTOR", "LIGAND", "DELTA"]
    MMdict = {}

    ##checking if the file has listed qualities:
    for i in range(len(mmpbsaread)):
        if componentname[0] in mmpbsaread[i]:
            titleline=i+4
        elif componentname[3] in mmpbsaread[i]:
            deltaline=i+4
            break
    #make empty dictionary
    for i in range(len(componentname)):
        MMdict[componentname[i]] = {}
    
    #make dictionary of the complex, receptor, ligand
    for i in range(titleline, titleline+9):
        splitrow = mmpbsaread[i].split()
        MMdict[componentname[0]][splitrow[0]] = float(splitrow[1])
        MMdict[componentname[1]][splitrow[0]] = float(splitrow[3])
        MMdict[componentname[2]][splitrow[0]] = float(splitrow[5])
    
    #make dictionary of the delta
    for i in range(deltaline,deltaline+9): #till the end
        splitrow = mmpbsaread[i].split()
        MMdict[componentname[3]][splitrow[0]] = splitrow[1]
    
    #ensure closing the file 
    mmpbsafile.close()
 
    return MMdict


In [23]:
#testing if it works# 
bound = makedict("/Users/otran/Desktop/mmpbsa2lie/poulosVMMPBSA/stats421_protein0100frames")
free = makedict("/Users/otran/Desktop/mmpbsa2lie/poulosVMMPBSA/stats421_100frames")

In [25]:
free

{'COMPLEX': {'ELE': -6898.0,
  'VDW': -4264.58,
  'INT': 19774.91,
  'GAS': 8612.32,
  'GBSUR': 167.72,
  'GB': -2368.98,
  'GBSOL': -2201.26,
  'GBELE': -9266.98,
  'GBTOT': 6411.06},
 'RECEPTOR': {'ELE': -6818.19,
  'VDW': -4226.27,
  'INT': 19710.61,
  'GAS': 8666.15,
  'GBSUR': 168.64,
  'GB': -2416.31,
  'GBSOL': -2247.67,
  'GBELE': -9234.5,
  'GBTOT': 6418.48},
 'LIGAND': {'ELE': 6.08,
  'VDW': 7.24,
  'INT': 64.29,
  'GAS': 77.61,
  'GBSUR': 2.83,
  'GB': -39.27,
  'GBSOL': -36.43,
  'GBELE': -33.18,
  'GBTOT': 41.18},
 'DELTA': {'ELE': '-85.89',
  'VDW': '-45.55',
  'INT': '0.00',
  'GAS': '-131.44',
  'GBSUR': '-3.76',
  'GB': '86.60',
  'GBSOL': '82.84',
  'GBELE': '0.71',
  'GBTOT': '-48.60'}}

In [26]:
bound

{'COMPLEX': {'ELE': -6898.0,
  'VDW': -4264.58,
  'INT': 19774.91,
  'GAS': 8612.32,
  'GBSUR': 167.72,
  'GB': -2368.98,
  'GBSOL': -2201.26,
  'GBELE': -9266.98,
  'GBTOT': 6411.06},
 'RECEPTOR': {'ELE': 6.08,
  'VDW': -4264.58,
  'INT': 19774.91,
  'GAS': 15516.41,
  'GBSUR': 167.72,
  'GB': -33.54,
  'GBSOL': 134.18,
  'GBELE': -27.45,
  'GBTOT': 15650.59},
 'LIGAND': {'ELE': 6.08,
  'VDW': 7.24,
  'INT': 64.29,
  'GAS': 77.61,
  'GBSUR': 2.83,
  'GB': -39.27,
  'GBSOL': -36.43,
  'GBELE': -33.18,
  'GBTOT': 41.18},
 'DELTA': {'ELE': '-6910.17',
  'VDW': '-7.24',
  'INT': '-64.29',
  'GAS': '-6981.70',
  'GBSUR': '-2.83',
  'GB': '-2296.17',
  'GBSOL': '-2299.01',
  'GBELE': '-9206.34',
  'GBTOT': '-9280.70'}}